In [ ]:
#install libraries
!pip install tensorflow-gpu==1.15.2
!pip install keras==2.0.8
!pip install python-chess==0.25.1 
!pip install profilehooks
!pip install numpy
!pip install pyperclip
!pip install h5py

In [6]:
#import libraries
import tensorflow
import keras
import chess
import numpy as np
import h5py

In [7]:
squares_index = {
  'a': 0,
  'b': 1,
  'c': 2,
  'd': 3,
  'e': 4,
  'f': 5,
  'g': 6,
  'h': 7
}

def square_to_index(square):
    letter = chess.square_name(square)
    return 8 - int(letter[1]), squares_index[letter[0]]

def split_dims(board):
  # this is the 3d matrix
    board3d = np.zeros((14, 8, 8), dtype=np.int8)

  # here we add the pieces's view on the matrix
    for piece in chess.PIECE_TYPES:
        for square in board.pieces(piece, chess.WHITE):
            idx = np.unravel_index(square, (8, 8))
            board3d[piece - 1][7 - idx[0]][idx[1]] = 1
        for square in board.pieces(piece, chess.BLACK):
            idx = np.unravel_index(square, (8, 8))
            board3d[piece + 5][7 - idx[0]][idx[1]] = 1

  # add attacks and valid moves too
  # so the network knows what is being attacked
    aux = board.turn
    board.turn = chess.WHITE
    for move in board.legal_moves:
        i, j = square_to_index(move.to_square)
        board3d[12][i][j] += 1
    board.turn = chess.BLACK
    for move in board.legal_moves:
        i, j = square_to_index(move.to_square)
        board3d[13][i][j] += 1
    board.turn = aux
    return board3d
  

In [8]:
def BoardBeforeMove(board, move):
  dims = split_dims(board)
  UCI = chess.Move.uci(board.parse_san(move))
  board.push_uci(UCI)
  return dims, UCI

In [9]:
def DataLoad():
  import pandas as pd
  dataframe = pd.read_csv('games.csv')
  dataframe = dataframe[dataframe['victory_status']== 'mate']
  dataframe_black = dataframe[dataframe['winner'] == 'black']
  dataframe_white = dataframe[dataframe['winner'] == 'white']
  return np.array(dataframe_white['moves']), np.array(dataframe_black['moves'])




In [19]:
def moves(game, type):
  Steps = game.split(' ')
  board = chess.Board()
  game = []
  moves = []
  i = 0
  for step in Steps:
    if (type == "white"):
      if (i % 2) == 0: 
        dims, UCI = BoardBeforeMove(board,step)
        moves.append(UCI)
        game.append(dims)
      else:
        a, b = BoardBeforeMove(board,step)
    else:
      if (i % 2) != 0:
        dims, UCI = BoardBeforeMove(board,step)
        moves.append(UCI)
        game.append(dims)
      else:
        a, b = BoardBeforeMove(board,step)
    i += 1
  return np.array(game), np.array(moves)


In [20]:
def GetData():
  WhiteWin = []
  WhiteMoves = []
  BlackWin = []
  BlackMoves = []
  wgames, bgames = DataLoad()
  for game in wgames:
    WW, WM = moves(game, "white")
    WhiteWin.append(WW)
    WhiteMoves.append(WM)
  for game in bgames:
    BW, BM = moves(game, "black")
    BlackWin.append(BW)
    BlackMoves.append(BM)
  
  return WhiteWin, WhiteMoves, BlackWin, BlackMoves

WX,WY,BX,BY = GetData()


In [26]:
print(WX[0][0])
print(WY[0][0])

[[[0 0 0 0 0 0 0 0]
  [0 0 0 0 0 0 0 0]
  [0 0 0 0 0 0 0 0]
  [0 0 0 0 0 0 0 0]
  [0 0 0 0 0 0 0 0]
  [0 0 0 0 0 0 0 0]
  [1 1 1 1 1 1 1 1]
  [0 0 0 0 0 0 0 0]]

 [[0 0 0 0 0 0 0 0]
  [0 0 0 0 0 0 0 0]
  [0 0 0 0 0 0 0 0]
  [0 0 0 0 0 0 0 0]
  [0 0 0 0 0 0 0 0]
  [0 0 0 0 0 0 0 0]
  [0 0 0 0 0 0 0 0]
  [0 1 0 0 0 0 1 0]]

 [[0 0 0 0 0 0 0 0]
  [0 0 0 0 0 0 0 0]
  [0 0 0 0 0 0 0 0]
  [0 0 0 0 0 0 0 0]
  [0 0 0 0 0 0 0 0]
  [0 0 0 0 0 0 0 0]
  [0 0 0 0 0 0 0 0]
  [0 0 1 0 0 1 0 0]]

 [[0 0 0 0 0 0 0 0]
  [0 0 0 0 0 0 0 0]
  [0 0 0 0 0 0 0 0]
  [0 0 0 0 0 0 0 0]
  [0 0 0 0 0 0 0 0]
  [0 0 0 0 0 0 0 0]
  [0 0 0 0 0 0 0 0]
  [1 0 0 0 0 0 0 1]]

 [[0 0 0 0 0 0 0 0]
  [0 0 0 0 0 0 0 0]
  [0 0 0 0 0 0 0 0]
  [0 0 0 0 0 0 0 0]
  [0 0 0 0 0 0 0 0]
  [0 0 0 0 0 0 0 0]
  [0 0 0 0 0 0 0 0]
  [0 0 0 1 0 0 0 0]]

 [[0 0 0 0 0 0 0 0]
  [0 0 0 0 0 0 0 0]
  [0 0 0 0 0 0 0 0]
  [0 0 0 0 0 0 0 0]
  [0 0 0 0 0 0 0 0]
  [0 0 0 0 0 0 0 0]
  [0 0 0 0 0 0 0 0]
  [0 0 0 0 1 0 0 0]]

 [[0 0 0 0 0 0 0 0]
  [1 1 1